# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning', 'R. E. Hviding']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

E. Bañados  ->  E. Bañados  |  ['E. Bañados']
Y. Wu  ->  Y. Wu  |  ['Y. Wu']
Y. Wu  ->  Y. Wu  |  ['Y. Wu']
S. Belladitta  ->  S. Belladitta  |  ['S. Belladitta']
F. Walter  ->  F. Walter  |  ['F. Walter']
J.-U. Pott  ->  J.-U. Pott  |  ['J.-U. Pott']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
I. J. M. Crossfield  ->  I. J. M. Crossfield  |  ['I. J. M. Crossfield']
K. Doi  ->  K. Doi  |  ['K. Doi']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
I. Almeida  ->  I. Almeida  |  ['I. Almeida']


S. K  ->  S. K. Krieger  |  ['S. K']
A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
R. E. Hviding  ->  R. E. Hviding  |  ['R. E. Hviding']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
Arxiv has 77 new papers today
          11 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/11 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2602.04974


extracting tarball to tmp_2602.04974... done.


/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2602.04974/cluster_main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'authors_list' from 'tmp_2602.04974/authors_list.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


Retrieving document from  https://arxiv.org/e-print/2602.04979


...

Issues with the citations
Too many commas in 'OzGrav, DWF (Deeper, Wider, Faster program), AST3,'
Retrieving document from  https://arxiv.org/e-print/2602.05989
extracting tarball to tmp_2602.05989...

 done.
Retrieving document from  https://arxiv.org/e-print/2602.06024


extracting tarball to tmp_2602.06024... done.


Found 104 bibliographic references in tmp_2602.06024/lrd_h2o_nat.bbl.
Issues with the citations
syntax error in line 5: unbalanced braces


/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex_bib.py:132: UserWarning: Error processing bibitem
 item = \bibitem[\protect\citeauthoryear{{Kocevski} et~al.}{2023}]{Kocevski2023}\begin{barticle}\bauthor{\bsnm{{Kocevski}}, \binits{D.D.}},\bauthor{\bsnm{{Onoue}}, \binits{M.}},\bauthor{\bsnm{{Inayoshi}}, \binits{K.}},\bauthor{\bsnm{{Trump}}, \binits{J.R.}},\bauthor{\bsnm{{Arrabal Haro}}, \binits{P.}},\bauthor{\bsnm{{Grazian}}, \binits{A.}},\bauthor{\bsnm{{Dickinson}}, \binits{M.}},\bauthor{\bsnm{{Finkelstein}}, \binits{S.L.}},\bauthor{\bsnm{{Kartaltepe}}, \binits{J.S.}},\bauthor{\bsnm{{Hirschmann}}, \binits{M.}},\bauthor{\bsnm{{Aird}}, \binits{J.}},\bauthor{\bsnm{{Holwerda}}, \binits{B.W.}},\bauthor{\bsnm{{Fujimoto}}, \binits{S.}},\bauthor{\bsnm{{Juneau}}, \binits{S.}},\bauthor{\bsnm{{Amor{í}n}}, \binits{R.O.}},\bauthor{\bsnm{{Backhaus}}, \binits{B.E.}},\bauthor{\bsnm{{Bagley}}, \binits{M.B.}},\bauthor{\bsnm{{Barro}}, \binits{G.}},\bauthor{\bsnm{{B

### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.04974-b31b1b.svg)](https://arxiv.org/abs/2602.04974) | **Clustering of z~6.6 Quasars and [O III] Emitters Constrains Host Halo Masses and Duty Cycles in 25 ASPIRE Fields**  |
|| J. Huang, et al. -- incl., <mark>E. Bañados</mark>, <mark>Y. Wu</mark> |
|*Appeared on*| *2026-02-06*|
|*Comments*| *23 pages, 10 figures, submitted to MNRAS*|
|**Abstract**|            We use data from the JWST ASPIRE Wide Field Slitless Spectroscopy (WFSS) program to measure the auto-correlation function of [O,{\sc iii}] emitters at 5.3$<z<$7.0 and the quasar--[O,{\sc iii}] emitter cross-correlation around 25 ASPIRE quasars (6.51$<z<$6.82; $\langle z\rangle=6.6$). We use synthetic source injection to calibrate the selection function, which we combine with the large-volume FLAMINGO-10k simulation (2.8,cGpc box) to construct realistic mock observations. Our simulation-based approach captures nonlinear structure growth and scale-dependent bias on small scales and derives covariance matrices that include cosmic variance. The clustering yields correlation lengths of $r_0^{\rm GG}=4.7^{+0.4}{-0.5},h^{-1}$,cMpc for the [O,{\sc iii}] auto-correlation with fixed slope $\gamma{\rm GG}=1.8$, and $r_0^{\rm QG}=8.7^{+0.8}{-0.9},h^{-1}$,cMpc for the quasar--[O,{\sc iii}] cross-correlation with $\gamma{\rm QG}=2.0$. We infer $\log(M_{h,{\rm min}}^{[{\rm O,III}]}/M_\odot)=10.5^{+0.1}{-0.1}$ for [O,{\sc iii}] emitters and $\log(M{h,{\rm min}}^{\rm QSO}/M_\odot)=12.1^{+0.3}{-0.4}$ for quasars. These imply duty cycles of $2.5^{+1.0}{-0.8}$,per,cent for [O,{\sc iii}] emitters and $0.3^{+4.0}{-0.3}$,per,cent for quasars, corresponding to UV-bright lifetimes of $t{\rm Q}=2.6^{+30}_{-2.5}$,Myr (less than 10,per,cent of a Salpeter $e$-folding time). The results indicate that the observed UV-luminous phase contributes little to total SMBH growth, placing tight constraints on early black-hole formation.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.04979-b31b1b.svg)](https://arxiv.org/abs/2602.04979) | **ASPIRE: The Environments and Dark Matter Halos of Luminous Quasars in the Epoch of Reionization**  |
|| F. Wang, et al. -- incl., <mark>Y. Wu</mark>, <mark>S. Belladitta</mark>, <mark>F. Walter</mark> |
|*Appeared on*| *2026-02-06*|
|*Comments*| *submitted to AAS journal in Nov. 2025*|
|**Abstract**|            We present a systematic study of the environments of 25 luminous quasars at $z > 6.5$ from the ASPIRE program. Using JWST/NIRCam WFSS data, we identified 487 galaxies at $5.3 \lesssim z \lesssim 7.0$ exhibiting [OIII] emission. Among these, 122 [OIII] emitters lie within $|\Delta v_{\rm los}| < 1000~{\rm km~s^{-1}}$ of the quasars, corresponding to a $\sim9.4$-fold enhancement relative to the average galaxy density at other redshifts. Furthermore, we identified 16 [CII]-emitting galaxies at the quasar redshifts from ALMA mosaic observations. A cross-correlation function (CCF) analysis between quasars and [OIII]+[CII] emitters yields a cross-correlation length of $r_0^{\rm QG} = 8.68^{+0.51}_{-0.55}~h^{-1}~\mathrm{cMpc}$ and a auto-correlation of $r_0^{\rm{QQ}}=15.76^{+2.48}_{-2.70}~h^{-1}~{\rm cMpc}$, indicating that $z \sim 7$ quasars reside in dark matter halos with $M_{\rm halo} = 10^{12.27^{+0.21}_{-0.26}}~M_\odot$. Notably, the number of [OIII]-emitting galaxies at quasar redshifts varies significantly from field to field, ranging from zero to twenty, highlighting a diverse quasar environment. Remarkably, seven quasars trace significant galaxy overdensities (i.e., protoclusters), with $\delta_{\rm gal} > 5$ within a volume of $V \sim 500~{\rm cMpc^3}$. We also find that $|\Delta v_{\rm los}|$ increases rapidly toward smaller galaxy-quasar separations in protocluster fields, consistent with galaxy kinematics around extremely massive halos in cosmological simulations. By combining JWST and ALMA data, we reveal the complex and diverse environments of these early quasars, providing robust evidence that the earliest luminous quasars are effective tracers of galaxy overdensities, albeit with substantial field-to-field variation.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.05094-b31b1b.svg)](https://arxiv.org/abs/2602.05094) | **Shocks, Winds, and a Torus: The Large Binocular Telescope Interferometer (LBTI) Resolves the Active Nucleus of NGC 4151**  |
|| J. W. Isbell, et al. -- incl., <mark>J.-U. Pott</mark> |
|*Appeared on*| *2026-02-06*|
|*Comments*| *Accepted for publication in ApJ. 13 pages, 6 figures*|
|**Abstract**|            We present mid-infrared (MIR) observations of the Seyfert 1 galaxy NGC 4151 using the Large Binocular Telescope Interferometer (LBTI). We took open-loop Fizeau images with 66-104 mas (5.8-9.1 pc) resolution in the N-band (at $8.7$ and $10.5~\mu m$), using the full resolution of the LBTI -- equivalent to that of a 28.8 m telescope. These images were complemented by AO imaging in the LM-bands ($3.7$ and $4.8~\mu m$), with 50-62 mas (4.4-5.4 pc) resolution. These images bridge the scales between previous Very Large Telescope Interferometer (VLTI)/MIDI and VLT/VISIR data, delivering ELT-like imaging resolution in the N-band. We resolve a dusty torus, (diameter 32 pc, PA$=125^{\circ}$), and detect dusty clouds within the narrow line region. Matching the resolution across four bands, we measured spatially-resolved SEDs of the central $\sim 100$ pc. Modified blackbody fitting revealed dust temperature and extinction profiles, indicating both heating from the accretion disk and additional shock heating due to the radio jet. The spatial coincidence of ionized emission (e.g., [Fe II] and [O III]), extended MIR structures, and radio features further supports the interpretation of shock heating. Comparison with NGC 1068 tests the Unified Model of Active Galactic Nuclei (Unified Model of AGN): Structures are similar, despite differences in orientation and Eddington ratio. NGC 4151's torus is smaller than NGC 1068's following a $r\propto L^{0.5}$ scaling. These thirty-meter-telescope class observations of NGC 4151 and NGC 1068 highlight the need to revise MIR radiative transfer models of AGN to account for jet-related heating.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.05954-b31b1b.svg)](https://arxiv.org/abs/2602.05954) | **Tracing AGN Feedback Power with Cool/Warm Outflow Densities: Predictions and Observational Implications**  |
|| <mark>I. Almeida</mark>, T. Costa, C. M. Harrison, S. R. Ward |
|*Appeared on*| *2026-02-06*|
|*Comments*| *24 pages, 18 figures. Accepted for publication in MNRAS*|
|**Abstract**|            Winds launched at the scale of the accretion disc or dusty torus in Active Galactic Nuclei (AGN) are thought to drive energy-conserving outflows that shape galaxy evolution. The key signature of such outflows, the presence of a hot ($T \gtrsim 10^9 \, \rm K$), shocked wind component, is hard to detect directly. Observations of AGN outflows typically probe a separate outflow phase: cool/warm gas with $T \lesssim 10^5 \, \rm K$. Here, we show that the density of cool outflowing gas scales with AGN luminosity, serving as an indirect diagnostic of the elusive hot, shocked wind. We use hydrodynamic simulations with the moving-mesh code AREPO to target the interaction between a small-scale AGN wind of speed $\approx 10^4 \, \rm km \, s^{-1}$ and galactic discs containing an idealised, clumpy interstellar medium (ISM). Through a new refinement scheme targeting rapidly-cooling, fast-moving gas, our simulations reach a resolution of $\lesssim 0.1 \, \rm pc$ in the cool, outflowing phase. We extract an ensemble of cool clouds from the AGN-driven outflows produced in our simulations, finding that their densities increase systematically with AGN wind power and AGN luminosity. Moreover, the mass distribution and internal properties of these cloudlets appear to be insensitive to the initial properties of the ISM, and shaped mainly by the dynamics of radiative, turbulent mixing layers. The increase in cool outflow density with kinetic wind power and AGN luminosity has profound implications for observational estimates of outflow rates and their scaling with AGN luminosity. Depending on the available outflow and density tracers, observationally-derived outflow rates may be overestimated by orders of magnitude.         |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.05168-b31b1b.svg)](https://arxiv.org/abs/2602.05168) | **Thermal Eclipse Observation of the Young Hot Neptune AU Mic b with Spitzer**  |
|| K. I. Collins, et al. -- incl., <mark>I. J. M. Crossfield</mark> |
|*Appeared on*| *2026-02-06*|
|*Comments*| *Submitted to AAS Journals, 29 pages, 34 figures, 6 tables*|
|**Abstract**|            We present the observation of a secondary eclipse of the young hot Neptune, AU Mic b, in the infrared using the Spitzer Space Telescope. Using a primary transit from Spitzer to constrain the system parameters, we tentatively detect an eclipse centered at $BJD=2458740.848893^{+0.00010}_{-0.000099}$ with an observed depth of $171\pm{29}$ ppm given an uninformed prior. This corresponds to a dayside brightness temperature of $T=1031\pm{58}$ K, which exceeds the calculated equilibrium temperature of $606\pm{19}$ K. We explore some possible explanations for these results, including inefficient heat redistribution, gravitational contraction, stellar pulsations, instrument systematics and choice of eclipse depth prior, but find none of these to be likely explanations for the observed eclipse parameters. We also explore the impact of correlated noise in the systematic trends, and we find that splitting the systematics into low-pass (smoothing) and high-pass trends is required to reach an optimal minimization of the low-frequency systematics in the resulting detrended light curve. Future observations with JWST are needed to confirm our eclipse detection with Spitzer.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.05247-b31b1b.svg)](https://arxiv.org/abs/2602.05247) | **ALMA 873 $μ$m Polarization Observations of the PDS~70 Disk**  |
|| H. B. Liu, et al. -- incl., <mark>K. Doi</mark> |
|*Appeared on*| *2026-02-06*|
|*Comments*| *Accepted to A&A. A more thorough instruction of the MCRT+MCMC fitting procedure is in the preprint here (this https URL)*|
|**Abstract**|            At a 112.4 pc distance, the PDS70 protoplanetary disk is a rare case that has been confirmed to host two accreting planets. This makes it the most important laboratory for studying dust growth in the context of planet formation. Here we present the first deep, full polarization observations at 873 $\mu$m wavelength. We detected $\sim$1%-2.5% linear polarization over the bulk of the $\sim$55-100 AU (sub)millimeter ring. The polarization position angles align preferentially with the projected minor axis of the disk. The standard interpretation is that the observed polarization is caused by dust self-scattering, with a maximum dust grain size of $\sim$100 $\mu$m. On $\gtrsim$10 AU scales, which can be resolved by the presented 873-3075 $\mu$m observations, the ring is marginally optical thick at 873 $\mu$m wavelength. Using Monte Carlo radiative transfer simulations, we found that an azimuthally asymmetric, marginally optically thick ring with a maximum dust grain size of $\sim$87 $\mu$m can reproduce the observed 873 $\mu$m polarization position angles and percentages. This study indicates that the coagulation of ice-coated dust in the protoplanetary disk may be limited by fragmentation or bouncing.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.05404-b31b1b.svg)](https://arxiv.org/abs/2602.05404) | **The Double-Burst Nature and Early Afterglow Evolution of Long GRB 110801A**  |
|| Q.-L. Wang, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2026-02-06*|
|*Comments*| *16 pages, 12 figures. arXiv admin note: text overlap with arXiv:2501.04906*|
|**Abstract**|            We present a comprehensive temporal and spectral analysis of the long-duration gamma-ray burst GRB 110801A, utilizing multi-band data from the Neil Gehrels Swift Observatory and ground-based telescopes. The $\gamma$-ray emission exhibits a distinct two-episode (``double-burst'') structure. Rapid follow-up observations in the optical and X-ray bands provide full coverage of the second burst. The optical light curve begins to rise approximately 135 s after the trigger, significantly preceding the second emission episode observed in X-rays and $\gamma$-rays at $\sim 320$ s. This chromatic behavior suggests different physical origins for the optical and high-energy emissions. Joint broadband spectral fitting (optical to $\gamma$-rays) during the second episode reveals that a two-component model, consisting of a power-law plus a Band function, provides a superior fit compared to single-component models. We interpret the power-law component as the afterglow of the first burst (dominating the optical band), while the Band component is attributed to the prompt emission of the second burst (dominating the high-energy bands). A physical synchrotron model is also found to be a viable candidate to explain the high-energy emission. Regarding the afterglow, the early optical light curve displays a sharp transition from a rise of $\sim t^{2.5}$ to $\sim t^{6.5}$, which is well-explained by a scenario involving both reverse shock (RS) and forward shock (FS) components. We constrain the key physical parameters of the burst, deriving an initial Lorentz factor $\Gamma_0 \sim 60$, a jet half-opening angle $\theta_j \sim 0.09$, and an isotropic kinetic energy $E_{\rm k,iso} \sim 10^{54.8}$ erg.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.05587-b31b1b.svg)](https://arxiv.org/abs/2602.05587) | **Exploring the central engines of gamma-ray bursts from prompt light curves**  |
|| <mark>X. Zhang</mark>, et al. |
|*Appeared on*| *2026-02-06*|
|*Comments*| *14pages, 6 figures and 1 table; accepted by Astronomy and Astrophysics*|
|**Abstract**|            Hyperaccreting stellar-mass black hole systems are leading candidates for the central engines of gamma-ray bursts (GRBs). Their jets are thought to be powered by either the Blandford-Znajek (BZ) process or neutrino-dominated accretion flows (NDAFs), but discriminating between these mechanisms remains challenging. To address this, we propose using the luminosity decay slope (parameter d) of GRB light curves to distinguish between the BZ and NDAF mechanisms, thereby linking the light-curve morphology to the central engine physics. By analysing 85 single-peaked GRBs with fast-rise, exponential-decay (FRED) profiles observed by Swift/BAT using 64 ms background-subtracted light curves, we fit the decay slope (parameter d) with the empirical Kocevski-Ryde-Liang (KRL) function and compare the results with theoretical predictions for the BZ (d approximately 1.67) and the NDAF (d approximately 3.7 to 7.8) mechanisms. We find that the decay slope (parameter d) can differentiate central engine mechanisms, with 15 GRBs consistent with the BZ mechanism and 22 supporting the NDAF mechanism. However, most events exhibit slopes within the range between 2 and 4, suggesting a hybrid of mechanisms, with NDAF being dominant.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.05989-b31b1b.svg)](https://arxiv.org/abs/2602.05989) | **Time lags and their association with the Boundary Layer structure in a Z source GX 349+2**  |
|| A. M.V.R., <mark>S. K</mark>, G. SD |
|*Appeared on*| *2026-02-06*|
|*Comments*| **|
|**Abstract**|            Studying the cross-correlation function between the soft and hard X-ray emission in Neutron Star Low Mass X-ray Binaries provides crucial insight into the structure and dynamics of the innermost accretion regions. In this work, we investigate the CCF of the Z-source GX 349+2 using an XMM-Newton observation. We noted that asymmetric CCFs with lags of a few hundred secondsbetween soft and hard band LCs in the horizontal branch, whereas CCFs remained symmetric in normal and flaring branches. We also performed a CCF study during the flux transition duration and observed lags of the order of a few tens to hundreds of seconds. Monte Carlo simulations were performed to assess the robustness of these CCFs, confirming their significance at a 95% confidence level. Spectral analysis during the flux transitions further suggests that the inner accretion disk extends close to the last stable orbit. We propose that the observed hard lags arise from the readjustment of the boundary layer/coronal region located near the inner edge of the accretion disk. From the measured lags, we estimate the characteristic size of the boundary layer. We show that the observed lags could also be associated with the depletion timescale of the boundary layer with low viscosity.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2602.04974/./figures/senslim_J0109m3047_new.png', 'tmp_2602.04974/./figures/senslim_J0224m4711_new.png', 'tmp_2602.04974/./figures/corr_chi_QG_GG_combined.png', 'tmp_2602.04974/./figures/quasar_lifetime_combined_lr.png']
copying  tmp_2602.04974/./figures/senslim_J0109m3047_new.png to _build/html/
copying  tmp_2602.04974/./figures/senslim_J0224m4711_new.png to _build/html/
copying  tmp_2602.04974/./figures/corr_chi_QG_GG_combined.png to _build/html/
copying  tmp_2602.04974/./figures/quasar_lifetime_combined_lr.png to _build/html/
exported in  _build/html/2602.04974.md
    + _build/html/tmp_2602.04974/./figures/senslim_J0109m3047_new.png
    + _build/html/tmp_2602.04974/./figures/senslim_J0224m4711_new.png
    + _build/html/tmp_2602.04974/./figures/corr_chi_QG_GG_combined.png
    + _build/html/tmp_2602.04974/./figures/quasar_lifetime_combined_lr.png
found figures ['tmp_2602.04979/./fig_quasar_Ncompanion_Delta_color.png', 'tmp_2602.04979/./fig_corrfunc_final_v20251023.

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\orcid}[1]{\ifthenelse{\equal{#1} }{\relax}{ \orcidlink{#1}}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$
$\newcommand{\oiii}{[O \textsc{iii}]}$
$\newcommand{\FeII}{Fe~{\sevenrm II}}$
$\newcommand{\FeIIf}{[Fe~{\sevenrm II}]}$
$\newcommand{\SIII}{[S~{\sevenrm III}]}$
$\newcommand{\HeI}{He~{\sevenrm I}}$
$\newcommand{\HeII}{He~{\sevenrm II}}$
$\newcommand{\NeV}{[Ne~{\sevenrm V}]}$
$\newcommand{\OIV}{[O~{\sevenrm IV}]}$
$\newcommand{\mpch}{~\mathrm{cMpc}~h^{-1}}$
$\newcommand{\rp}{r_{\rm p}}$
$\newcommand{\msun}{{\rm{M_{\odot}}}}$
$\newcommand{\mminq}{M^{\rm QSO}_{h, \rm min}}$
$\newcommand{\mming}{M^{\rm \oiii}_{h, \rm min}}$</div>



<div id="title">

# Clustering of $z\sim6.6$ Quasars and [O III] Emitters Constrains Host Halo Masses and Duty Cycles in 25 ASPIRE Fields

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2602.04974-b31b1b.svg)](https://arxiv.org/abs/2602.04974)<mark>Appeared on: 2026-02-06</mark> -  _23 pages, 10 figures, submitted to MNRAS_

</div>
<div id="authors">

J. Huang, et al. -- incl., <mark>E. Bañados</mark>, <mark>Y. Wu</mark>

</div>
<div id="abstract">

**Abstract:** We use data from the JWST ASPIRE Wide Field Slitless Spectroscopy (WFSS) program to measure the auto-correlation function of $\oiii$ -emitters at $5.3<z<7.0$ and the quasar-- $\oiii$ -emitter cross-correlation function around 25 ASPIRE quasars ( $6.51 < z < 6.82$ ; $\langle z\rangle = 6.6$ ).We use synthetic source injection to calibrate the selection function, which we combine with the large-volume cosmological simulation FLAMINGO-10k (2.8 cGpc box) to construct realistic mock observations. Our simulation-based approach offers two key advantages: (1) the clustering models capture nonlinear structure growth and scale-dependent bias on small scales beyond analytic prescriptions, and (2) covariance matrices from mock realizations account for cosmic variance and correlated uncertainties across radial bins.Our clustering measurements yield a correlation length of $r_0^{\rm GG} = 4.7^{+0.5}_{-0.6} \mpch$ for the $\oiii$ -emitter auto-correlation with a fixed power-law slope $\gamma_{\rm GG}=1.8$ ,and $r_0^{\rm QG} = 8.9^{+0.9}_{-1.0} \mpch$ for the quasar-- $\oiii$ -emitter cross-correlation with $\gamma_{\rm QG}=2.0$ .We jointly estimate the halo masses by assuming a step-function halo occupation distribution (HOD) with mass-dependent covariance matrices. Our analysis yields $\log (M_{h, \mathrm{min}}^{\mathrm{[OIII]}}/\msun) = 10.55^{+0.11}_{-0.12}$ for $\oiii$ -emitter host halos and $\log (M_{h, \mathrm{min}}^{\mathrm{QSO}}/\msun) = 12.13^{+0.31}_{-0.38}$ for quasar host halos. Based on these minimum halo masses, we estimate duty cycles of $2.5^{+1.3}_{-0.8}\%$ for $\oiii$ -emitters,and $0.3^{+4.8}_{-0.3}\%$ for quasars. The low quasar duty cycle implies UV-bright lifetimes of only $t_{\rm Q} =2.64^{+39.15}_{-2.61} \rm Myr$ , representing $\lesssim 10\%$ of a Salpeter $e$ -folding time. This indicates that the observed UV-luminous phase contributes minimally to total SMBH mass assembly, placing tight constraints on early black hole growth models.

</div>

<div id="div_fig1">

<img src="tmp_2602.04974/./figures/senslim_J0109m3047_new.png" alt="Fig7.1" width="50%"/><img src="tmp_2602.04974/./figures/senslim_J0224m4711_new.png" alt="Fig7.2" width="50%"/>

**Figure 7. -** Two examples of the sensitivity map for quasar field J0109-3047 and J0224-4711, the orange stars mark the location of the quasars. Each pixel value (\texttt{Flim}) on the sensitivity map is computed from the S/N of the injected source. The inject mock grid has size 40 rows $\times$ 25 columns, and the direction of the columns follows the direction of the dispersion. Grid points with no detection (i.e.,
    the redshift of the mock source is misidentified with $|z-z_{\rm true}|>0.01$), where $z_{\rm true}=6.5$ for the injected mocks, are shown in pink.
    In the left panel, we mark the dispersion direction of Grism R (dispersion across detector rows) as red arrows for modules A and B. The color bars correspond to the $5\sigma$ flux limit, where deeper bluer color means more sensitive to the faint sources. The vertical stripes of non-detection within the FOV correspond to the contamination due to the dispersed foreground bright sources. (*fig:sensitivity*)

</div>
<div id="div_fig2">

<img src="tmp_2602.04974/./figures/corr_chi_QG_GG_combined.png" alt="Fig8" width="100%"/>

**Figure 8. -** 
Correlation matrices of the volume-averaged correlation functions, normalized by their diagonal elements.
The left panel shows the auto-correlation function of [O iii] emitters, $\chi_{\rm GG}$, and the right panel shows the quasar–[O iii] cross-correlation function, $\chi_{\rm QG}$.
Both are computed using Eq. \ref{eq:covariance} based on 1{,}000 mock realizations generated for the minimum-mass model with
$\log(M_h^{\rm gal}/M_\odot)=10.6$ and $\log(M_h^{\rm QSO}/M_\odot)=12.2$, where one mock realization computes the correlation functions based on the total number counts across all 25 ASPIRE-like quasar fields.
The tick labels on each axis indicate the bin centers for the projected-radius, $r_p^{\rm center}$.
The off-diagonal structure reflects correlated uncertainties arising from pair-count covariance and large-scale structure modes coupling across bins.
 (*fig:covar_sim*)

</div>
<div id="div_fig3">

<img src="tmp_2602.04974/./figures/quasar_lifetime_combined_lr.png" alt="Fig11" width="100%"/>

**Figure 11. -** **_Left:**_ Redshift evolution of the quasar lifetime. The dashed curves show the lifetime with fixed quasar duty cycle, and the gray shaded region shows the forbidden region where $t_{\rm Q}>t_{\rm H}$ or, equivalently, $n_{\rm Q}>n_{\rm halo}$ where $n_{\rm halo}$ is the number density of quasar host halos.  The color bar shows the $M_{1450}$ magnitude of the quasar sample, with darker colors indicating lower luminosity.
    Our result for the ASPIRE sample is marked as the diamond data point. The measurement from $\rm Ly\alpha$ damping wings for individual quasars at $z>6$ are shown as the round  ([\vDurov\vc\'\ikov\'a, et. al 2024](https://ui.adsabs.harvard.edu/abs/2024ApJ...969..162D))  and square open makers  ([Davies, Hennawi and Eilers 2019](https://ui.adsabs.harvard.edu/abs/2019ApJ...884L..19D), [Wang, et. al 2020](https://ui.adsabs.harvard.edu/abs/2020ApJ...896...23W), [Yang, et. al 2020](https://ui.adsabs.harvard.edu/abs/2020ApJ...897L..14Y)) . For $z\gtrsim6$ quasars, we include the EIGER measurement, which uses $\oiii$-emitters as tracers to measure the quasar cross-correlation by putting together four quasar fields (shown as the star data point,   ([Eilers, et. al 2024](https://ui.adsabs.harvard.edu/abs/2024arXiv240307986E)) ). We also include measurements from [Pizzati, et. al (2024)](https://ui.adsabs.harvard.edu/abs/2024MNRAS.534.3155P) and [Arita, et. al (2024)](https://ui.adsabs.harvard.edu/abs/2024MNRAS.tmp.2641A). For lower redshift quasars, we include the clustering measurement from [Shen, et. al (2007)](https://ui.adsabs.harvard.edu/abs/2007AJ....133.2222S), [White, et. al (2012)](https://ui.adsabs.harvard.edu/abs/2012MNRAS.424..933W), [Eftekharzadeh, et. al (2015)](https://ui.adsabs.harvard.edu/abs/2015MNRAS.453.2779E), [Laurent, et. al (2017)](https://ui.adsabs.harvard.edu/abs/2017JCAP...07..017L), [Pizzati, et. al (2024)](https://ui.adsabs.harvard.edu/abs/2024MNRAS.528.4466P), [Giner Mascarell, Eilers and Storey-Fisher (2025)](https://ui.adsabs.harvard.edu/abs/2025arXiv251117413G). **_Right:**_ luminosity dependence of the quasar lifetime. The color bar shows the mean redshift of each sample.
     (*fig:fduty_z_m1450*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2602.04974"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\url}[1]{\href{#1}{#1}}$
$\newcommand{\dodoi}[1]{doi:~\href{http://doi.org/#1}{\nolinkurl{#1}}}$
$\newcommand{\doeprint}[1]{\href{http://ascl.net/#1}{\nolinkurl{http://ascl.net/#1}}}$
$\newcommand{\doarXiv}[1]{\href{https://arxiv.org/abs/#1}{\nolinkurl{https://arxiv.org/abs/#1}}}$
$\newcommand{\rqg}{r_0^{\rm QG} = 8.68^{+0.51}_{-0.55}~h^{-1}~\mathrm{cMpc}}$
$\newcommand{\mhalo}{M_{\rm halo} = 10^{12.27^{+0.21}_{-0.26}}~M_\odot}$
$\newcommand\natexlab{#1}$</div>



<div id="title">

# ASPIRE: The Environments and Dark Matter Halos of Luminous Quasars in the Epoch of Reionization

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2602.04979-b31b1b.svg)](https://arxiv.org/abs/2602.04979)<mark>Appeared on: 2026-02-06</mark> -  _submitted to AAS journal in Nov. 2025_

</div>
<div id="authors">

F. Wang, et al. -- incl., <mark>E. Bañados</mark>, <mark>Y. Wu</mark>, <mark>S. Belladitta</mark>, <mark>F. Walter</mark>

</div>
<div id="abstract">

**Abstract:** We present a systematic study of the environments of 25 luminous quasars at $z > 6.5$ from the ASPIRE program. Using JWST/NIRCam WFSS data, we identified 487 galaxies at $5.3 \lesssim z \lesssim 7.0$ exhibiting [ $\ion{O}{3}$ ] emission. Among these, 122 [ $\ion{O}{3}$ ] emitters lie within $|\Delta v_{\rm los}| < 1000 \mathrm{km s^{-1}}$ of the quasars, corresponding to a $\sim9.4$ -fold enhancement relative to the average galaxy density at other redshifts. Furthermore, we identified 16 [ $\ion{C}{2}$ ] -emitting galaxies at the quasar redshifts from ALMA mosaic observations. A cross-correlation function (CCF) analysis between quasars and [ $\ion{O}{3}$ ] + [ $\ion{C}{2}$ ] emitters yields a cross-correlation length of $r_0^{\rm QG} = 8.68^{+0.51}_{-0.55} h^{-1} \mathrm{cMpc}$ and a auto-correlation of $r_0^{\rm{QQ}}=15.76^{+2.48}_{-2.70} h^{-1} {\rm cMpc}$ , indicating that $z \sim 7$ quasars reside in dark matter halos with $M_{\rm halo} = 10^{12.27^{+0.21}_{-0.26}} M_\odot$ . Notably, the number of [ $\ion{O}{3}$ ] -emitting galaxies at quasar redshifts varies significantly from field to field, ranging from zero to twenty, highlighting a diverse quasar environment. Remarkably, seven quasars trace significant galaxy overdensities (i.e., protoclusters), with $\delta_{\rm gal} > 5$ within a volume of $V \sim 500 {\rm cMpc^3}$ . We also find that $|\Delta v_{\rm los}|$ increases rapidly toward smaller galaxy-quasar separations in protocluster fields, consistent with galaxy kinematics around extremely massive halos in cosmological simulations. By combining JWST and ALMA data, we reveal the complex and diverse environments of these early quasars, providing robust evidence that the earliest luminous quasars are effective tracers of galaxy overdensities, albeit with substantial field-to-field variation.

</div>

<div id="div_fig1">

<img src="tmp_2602.04979/./fig_quasar_Ncompanion_Delta_color.png" alt="Fig14" width="100%"/>

**Figure 14. -** ** Large variance of the number of quasar companion galaxies.**
 Galaxy numbers are determined by counting [$\ion${O}{3}] emitters within $\Delta |v_{\rm los}| < 1000 \mathrm{km s^{-1}}$ relative to the central quasars. The number of companion galaxies varies significantly from field to field, ranging from zero (lower than expected) to 20 (corresponding to $\delta_{\rm gal} \simeq 22$).
 (*fig:ngal*)

</div>
<div id="div_fig2">

<img src="tmp_2602.04979/./fig_corrfunc_final_v20251023.png" alt="Fig6" width="100%"/>

**Figure 6. -** **  Quasar-galaxy cross-correlation function.**
The orange points represent the measured quasar-galaxy cross-correlation function based on [$\ion${O}{3}] emitters while the dark red points denote the cross-correlation function from [$\ion${C}{2}] emitters.
The purple dashed line and dark (shallow) shaded region show the power-law fit when fixing the slope $\gamma_{\rm{QG}} = 2.0$ and its associated $1\sigma$($2\sigma$) posterior range. The blue dashed line show the predictions based on \texttt{Trinity} model. The gray dots are quasar-galaxy cross-correlation function measured by the EIGER program using only four quasars at $z\sim6$\citep{Eilers24}.
 (*fig:ccf*)

</div>
<div id="div_fig3">

<img src="tmp_2602.04979/./fig_m1450.png" alt="Fig1" width="100%"/>

**Figure 1. -** ** The redshift and absolute magnitude distribution of ASPIRE quasars.**
We selected a flux-limited sample of 25 quasars at $6.5<z\lesssim6.8$ with $M_{1450}<-25.0$(orange asterisks). To ensure all ASPIRE quasars can be observed with ALMA, we excluded known quasars with $\rm Decl. > 30^\circ$. This sample also includes a radio-loud quasar (red asterisk, $z=6.82$). The triangles denote three quasars satisfy our luminosity cut but were published after JWST Cycle 1 proposal deadline and therefore were not included in ASPIRE.
 (*fig:sample*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2602.04979"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\code}[1]{{\color{blue} \texttt{#1}}}$
$\newcommand{\package}[1]{{\color{pink} \texttt{#1}}}$
$\newcommand{\oiii}{\lbrackOIII\rbrack}$
$\newcommand{\feii}{\lbrackFeII\rbrack}$</div>



<div id="title">

# Shocks, Winds, and a Torus: The Large Binocular Telescope Interferometer (LBTI) Resolves the Active Nucleus of NGC 4151

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2602.05094-b31b1b.svg)](https://arxiv.org/abs/2602.05094)<mark>Appeared on: 2026-02-06</mark> -  _Accepted for publication in ApJ. 13 pages, 6 figures_

</div>
<div id="authors">

J. W. Isbell, et al. -- incl., <mark>J.-U. Pott</mark>

</div>
<div id="abstract">

**Abstract:** We present mid-infrared (MIR) observations of the Seyfert 1 galaxy NGC 4151 using the Large Binocular Telescope Interferometer (LBTI).We took open-loop Fizeau images with 66--104 mas (5.8--9.1 pc) resolution in the N-band (at $8.7$ and $10.5$  $\micron$ ), using the full resolution of the LBTI -- equivalent to that of a 28.8 m telescope. These images were complemented by AO imaging in the LM-bands ( $3.7$ and $4.8$  $\micron$ ), with 50--62 mas (4.4--5.4 pc) resolution.These images bridge the scales between previous Very Large Telescope Interferometer (VLTI)/MIDI and VLT/VISIR data, delivering ELT-like imaging resolution in the N-band.We resolve a dusty torus, (diameter 32 pc, PA $=125^{\circ}$ ), and detect dusty clouds within the narrow line region.Matching the resolution across four bands, we measured spatially-resolved SEDs of the central $\sim 100$ pc.Modified blackbody fitting revealed dust temperature and extinction profiles, indicating both heating from the accretion disk and additional shock heating due to the radio jet.The spatial coincidence of ionized emission (e.g., $\feii$ and $\oiii$ ), extended MIR structures, and radio features further supports the interpretation of shock heating.Comparison with NGC 1068 tests the Unified Model of Active Galactic Nuclei (Unified Model of AGN): Structures are similar, despite differences in orientation and Eddington ratio. NGC 4151’s torus is smaller than NGC 1068’s following a $r\propto L^{0.5}$ scaling.These ELT-like observations of NGC 4151 and NGC 1068 highlight the need to revise MIR radiative transfer models of AGN to account for jet-related heating.

</div>

<div id="div_fig1">

<img src="tmp_2602.05094/./nir_comparison.png" alt="Fig5" width="100%"/>

**Figure 5. -** Comparison of MIR and visible/NIR structures.
    The $\oiii$ image is from \citet{hutchings1999} and the other visible/NIR images are from \cite{may2020}. In the _top row_ we show $[\rm{O}{III}] \lambda501$ nm, H$\alpha \lambda656$ nm, $[\rm{Fe}{II}] \lambda1644$ nm, and $H_2 \lambda1748$ nm, from left to right. In the _bottom row_ we show contours of the same, with the LBTI 10.5 \micron contours overplotted (same levels as Fig. \ref{fig:deconv}). In the $[\rm{Fe}{II}] \lambda1644$ panel, we also show the radio 5 GHz emission from \cite{williams2020}. Many of the MIR features are coincident with visible/NIR emission features. The MIR arc in particular exhibits similarities to the $[\rm{O}III]$ and H$\alpha$ emission. The molecular hydrogen emission is nearly anti-coincident with the visible/MIR features. (*fig:nir*)

</div>
<div id="div_fig2">

<img src="tmp_2602.05094/./fizeau_psfs.png" alt="Fig1" width="100%"/>

**Figure 1. -** LBTI/NOMIC Fizeau images of NGC 4151 and PSF calibrator HD105140. The left two panels (in blue) show the 8.7 \micron images of the sources after frame selection, corotation, and stacking. The right two panels (in red) show the same at 10.5 \micron. The PSF calibrator has been artificially rotated to match the observed position angles of the target. Inset in each panel is the median Fizeau PSF without corotation; it shows the characteristic fringe pattern and the quality of the frame selection. Extended emission is immediately visible in the NGC 4151 images; the images are much more filled in with extended flux than the calibrators. (*fig:psfs_fizeau*)

</div>
<div id="div_fig3">

<img src="tmp_2602.05094/./ao_psfs.png" alt="Fig2" width="100%"/>

**Figure 2. -** LBTI/LMIRCam AO images of NGC 4151 and PSF calibrator NGP 39 74. The left two panels (in viridis) show the 3.7 \micron images of the sources after frame selection, corotation, and stacking. The right two panels (in magma) show the same at 4.8 \micron. The PSF calibrator has been artificially rotated to match the observed position angles of the target. Extended emission is visible near the cores of the NGC 4151 images relative to the calibrator. (*fig:psfs_ao*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2602.05094"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\working}{$
$\begin{wrapfigure}{l}{0.25\textwidth}$
$\vspace{-20pt}$
$\begin{center}$
$\includegraphics[width=0.25\textwidth]{figures/working-animal.jpg}$
$\end{center}$
$\vspace{-10pt}$
$\end{wrapfigure}$
$}$
$\newcommand{\continue}{\todo[inline,color=Orange]{$
$\centerline{ \textbf{\Large CONTINUE} }$
$} }$
$\newcommand{\fermi}{\emph{Fermi}}$
$\newcommand{\xmark}{\ding{56}}$
$\newcommand{\ok}{\ding{52}}$
$\newcommand{\grt}{GM/c^3}$
$\newcommand{\eg}[1]{(e.g. \citealt{#1})}$
$\newcommand{\orcid}[1]{\href{#1}{\includegraphics[scale=0.04]{figures/orcid.png}}}$
$\newcommand{\lagn}{L_{\rm AGN}}$
$\newcommand{\ledd}{L_{\rm Edd}}$
$\newcommand{\medd}{\dot{M}_{\rm Edd}}$
$\newcommand{\msun}{{\rm M_\odot}}$
$\newcommand{\harm}{\texttt{harm }}$
$\newcommand{\hamr}{\texttt{H-AMR }}$
$\newcommand{\arepo}{\texttt{AREPO }}$
$\newcommand{\mbh}{M_{\rm BH}}$
$\newcommand{\kms}{km s^{-1} }$
$\newcommand{\ergs}{erg s^{-1} }$
$\newcommand{\dens}{g cm^{-3} }$
$\newcommand{\numberdens}{cm^{-3} }$
$\newcommand{\mach}{\mathcal{M}}$
$\newcommand{\Ward}{\textit{W24} }$
$\newcommand{\Wardf}{\textit{W24}}$
$\newcommand\code{#1}$
$\newcommand\todoi{#1}$
$\newcommand\ivan{#1}$
$\newcommand\tiago{#1}$
$\newcommand\chris{#1}$
$\newcommand\rst{#1}$</div>



<div id="title">

# Tracing AGN Feedback Power with Cool/Warm Outflow Densities: Predictions and Observational Implications

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2602.05954-b31b1b.svg)](https://arxiv.org/abs/2602.05954)<mark>Appeared on: 2026-02-06</mark> -  _24 pages, 18 figures. Accepted for publication in MNRAS_

</div>
<div id="authors">

<mark>I. Almeida</mark>, T. Costa, C. M. Harrison, S. R. Ward

</div>
<div id="abstract">

**Abstract:** Winds launched at the scale of the accretion disc or dusty torus in Active Galactic Nuclei (AGN) are thought to drive energy-conserving outflows that shape galaxy evolution.The key signature of such outflows, the presence of a hot ( $T \gtrsim 10^9   \rm K$ ), shocked wind component, is hard to detect directly. Observations of AGN outflows typically probe a separate outflow phase: cool/warm gas with $T \lesssim 10^5   \rm K$ .Here, we show that the density of cool outflowing gas scales with AGN luminosity, serving as an indirect diagnostic of the elusive hot, shocked wind.We use hydrodynamic simulations with the moving-mesh code \texttt{AREPO} to target the interaction between a small-scale AGN wind of speed $\approx 10^4   \rm km   s^{-1}$ and galactic discs containing an idealised, clumpy interstellar medium (ISM). Through a new refinement scheme targeting rapidly-cooling, fast-moving gas, our simulations reach a resolution of $\lesssim 0.1   \rm pc$ in the cool, outflowing phase.We extract an ensemble of cool clouds from the AGN-driven outflows produced in our simulations, finding that their densities increase systematically with AGN wind power and AGN luminosity. Moreover, the mass distribution and internal properties of these cloudlets appear to be insensitive to the initial properties of the ISM, and shaped mainly by the dynamics of radiative, turbulent mixing layers.The increase in cool outflow density with kinetic wind power and AGN luminosity has profound implications for observational estimates of outflow rates and their scaling with AGN luminosity. Depending on the available outflow and density tracers, observationally-derived outflow rates may be overestimated by orders of magnitude.

</div>

<div id="div_fig1">

<img src="tmp_2602.05954/./figures/L-hist-ML.png" alt="Fig18" width="100%"/>

**Figure 18. -** Examining the impact of AGN luminosity on the main characteristics of the outflowing CGCs. (_Top panel_) The density distributions from our highest resolution simulations for $\lagn  = 10^{43},  10^{45}, \text{ and } 10^{47}$ erg s$^{-1}$ at 1 Myr, showing a shift to higher values with increasing AGN power. The lower panels illustrate the average density of CGCs (left) and the average size of the clouds (right), as a function of the AGN bolometric luminosity for a range of simulations investigated in this work at $t = 1$ Myr (except for the simulations with metal-line cooling, which the time is the maximum simulation time shown in Table \ref{tab:simulations-params}). The main results, showed as the dark blue stars and black squares, refer to the highest resolution simulations group --with resolution boost factor $\beta = 64$.
    The shaded grey areas highlight the full range of values across all of the simulations (see Table \ref{tab:simulations-params}), plus an 1$\sigma$ error range from the main run. The average values from the different simulations are shown as individual horizontal lines as described in the legend. In the lower left panel, average density values for the CGC cores from the main simulations are also displayed. (_Lower Left Panel_): We can see the CGC's density increasing with $L_{\rm AGN}^{1/2}$, up to the breakpoint for $\lagn \geq 10^{46}$ erg s$^{-1}$. This trend is consistent across all our simulations. However, this breakpoint is not observed when only the high density cores are analyzed (see secion \ref{subsec:Impact-LAGN}). (_Lower Right Panel_): The average CGC sizes follow $L_{\rm AGN}^{-1/6}$.
    Our simulations set can be divided into Middle Resolution ($M_{\rm target} = 100  $\msun$$) and High Resolution ($M_{\rm target} = \lesssim 10  $\msun$$). The change in resolution did not affect the density results but did show a reduction in the average cloud size, up to the break at $\lagn = 10^{46}$ erg s$^{-1}$. Our results indicate that brighter AGNs result in smaller denser cool outflowing clouds. (*fig:L-hist*)

</div>
<div id="div_fig2">

<img src="tmp_2602.05954/./figures/pressure-rho-rad.png" alt="Fig7" width="100%"/>

**Figure 7. -** This figure includes all gas with $v_r > 10$ km s$^{-1}$, without applying the cloud selection process. The gas is divided into cool ($T \leq 2 \times 10^4$ K, blue), hot ($T > 2 \times 10^4$ K, dark red), and the "hottest" ($T > 10^7$ K, brown) --which is a subset of the hot phase-- phases. All simulations are analysed at the time when the reverse shock radius is approximately $r_{\rm RS} \approx 150$–$200$ pc, the time when each simulation was analysed is shown in the top x-axis of the upper panel.
    _Upper panel:_ Average hydrogen number density ($n_{\rm H}$) of each gas phase for $D_{\rm AGN} > r_{\rm RS}$. The cool phase line follows $\lagn^{s}$, with $s=0.5$, as we found from the cloud analysis (see Figure \ref{fig:L-hist}). The lines show a linear relation between $\lagn$ and $n_\mathrm{H}$ for the hottest phase and the hot phase up to $\lagn^{s} = 10^{46}$ erg s$^{-1}$.
    _Middle panel:_ Radial profile of hydrogen number density for the different phases. Line styles correspond to different AGN luminosities: $\lagn = 10^{44}$ erg s$^{-1}$(dashed), $\lagn = 10^{45}$ erg s$^{-1}$(dash-dotted), and $\lagn = 10^{46}$ erg s$^{-1}$(dotted). A density jump is visible in the hot phase at $r_{\rm RS}$, coinciding with the appearance of the cool gas component.
    _Lower panel:_ Radial volume-weighted pressure profiles for the simulations. A corresponding pressure jump at $r_{\rm RS}$ is also observed. The plot reveals an increasing pressure gradient between the cool and hot phases with rising $\lagn$, indicating a more underpressured cool phase in the presence of more luminous AGN.
     (*fig:pressure-radius*)

</div>
<div id="div_fig3">

<img src="tmp_2602.05954/./figures/Mdot-Edot-n.png" alt="Fig8" width="100%"/>

**Figure 8. -** _Top Panel:_ Fraction of the mass outflow rate for gases exceeding certain density thresholds. For instance, gas with densities above $10^3$ cm$^{-3}$ accounts for approximately 30\% of the total outflowing CGC mass at $\lagn = 10^{45}$ erg s$^{-1}$ and about 60\% at $\lagn = 10^{47}$ erg s$^{-1}$. _Middle Panel:_ Kinetic coupling energy of the outflowing CGCs, where the ratio $\dot{E}_k / $\lagn$$ is low, indicating that this gas phase does not carry a significant amount of the total energy. _Bottom Panels:_$\dot{M}_{\rm OF}$ and $\dot{E}_k/$\lagn$$ as function of AGN luminosity for the cool phase. (*fig:mdot_ek*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2602.05954"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\eazy}{\texttt{EAzY}}$
$\newcommand{\msae}{\texttt{msaexp}}$
$\newcommand{\prospector}{\texttt{Prospector}}$
$\newcommand{\cloudy}{\texttt{Cloudy}}$
$\newcommand{\dynesty}{\texttt{dynesty}}$
$\newcommand{\phoenix}{\texttt{Phoenix}}$
$\newcommand{\tlusty}{\texttt{tlusty}}$
$\newcommand{\prism}{\texttt{Prism}}$
$\newcommand{\msun}{{\rm M}_{\odot}}$
$\newcommand{\zsol}{{\rm Z}_{\odot}}$
$\newcommand{\lsun}{{\rm L}_\odot}$
$\newcommand{\mstar}{{\rm M}_{\star}}$
$\newcommand{\zphot}{z_{\rm phot}}$
$\newcommand{\zspec}{z_{\rm spec}}$
$\newcommand{\Av}{A_{\rm V}}$
$\newcommand{\rwater}{r_{\rm H_{2}O}}$
$\newcommand{\hi}{H \textsc{i}}$
$\newcommand{\hei}{He \textsc{i}}$
$\newcommand{\heii}{He \textsc{ii}}$
$\newcommand{\ha}{H\alpha}$
$\newcommand{\hb}{H\beta}$
$\newcommand{\pab}{Pa-\beta}$
$\newcommand{\pag}{Pa-\gamma}$
$\newcommand{\pad}{Pa-\delta}$
$\newcommand{\nii}{[N {\sc ii}]}$
$\newcommand{\oiii}{[O {\sc iii}]}$
$\newcommand{\cii}{[C {\sc ii}]}$
$\newcommand{\feii}{Fe {\sc ii}}$
$\newcommand{\hinf}{H_\infty}$
$\newcommand{\oi}{O {\sc i} \lambda8446}$
$\newcommand{\water}{\rm{H_{2}O}}$
$\newcommand{\niia}{[N {\sc ii}] \lambda6549}$
$\newcommand{\niib}{[N {\sc ii}] \lambda6585}$
$\newcommand{\oiiia}{[O {\sc iii}] \lambda4960}$
$\newcommand{\oiiib}{[O {\sc iii}] \lambda5008}$
$\newcommand{\heiiuv}{He \textsc{ii} \lambda1640}$
$\newcommand{\heiiir}{He \textsc{ii} \lambda4686}$
$\newcommand{\heia}{He \textsc{i} \lambda1.08\mum}$
$\newcommand{\heib}{He \textsc{i} \lambda1.28\mum}$
$\newcommand{\kms}{{\rm km s^{-1}}}$
$\newcommand{\cms}{{\rm cm s^{-2}}}$
$\newcommand{\cgs}{{\rm erg   s^{-1}   cm^{-2}   Å^{-1}}}$
$\newcommand{\teff}{T_{\rm eff}}$
$\newcommand{\met}{{\rm[M/H]}}$
$\newcommand{\logg}{{\rm log}(g)}$
$\newcommand{\qion}{Q_{\rm ion}}$
$\newcommand{\qionh}{Q_{\rm ion, H\textsc{i}}}$
$\newcommand{\qionhe}{Q_{\rm ion, He\textsc{ii}}}$
$\newcommand{\vturb}{v_{\rm turb}}$
$\newcommand{\rcloud}{R_{\rm cloud}}$
$\newcommand{\fagn}{f_{\rm AGN}}$
$\newcommand{\aion}{\alpha_{\rm ion}}$
$\newcommand{\bopt}{\beta_{\rm opt}}$
$\newcommand{\rdwide}{WIDE-EGS-2974}$
$\newcommand{\rdcapers}{CAPERS-UDS-23216}$
$\newcommand{\rdcapersegs}{CAPERS-EGS-212932}$
$\newcommand{\rduncover}{UNCOVER-A2744-20698}$
$\newcommand{\rdqso}{A2744-QSO1}$
$\newcommand{\rdjades}{JADES-GN-28074}$
$\newcommand{\rdsp}{WIDE-GN-7133}$
$\newcommand{\qg}{ZF-UDS-7329}$
$\newcommand{\galjades}{JADES-138717}$
$\newcommand{\galceers}{CEERS-75915}$
$\newcommand{\sfit}{{\texttt{unite}}}$
$\newcommand{\actaa}{Acta Astron.}$
$\newcommand{\araa}{Annu. Rev. Astron. Astrophys.}$
$\newcommand{\areps}{Annu. Rev. Earth Planet. Sci.}$
$\newcommand{\aar}{Astron. Astrophys. Rev.}$
$\newcommand{\ab}{Astrobiology}$
$\newcommand{\aj}{Astron. J.}$
$\newcommand{\ac}{Astron. Comput.}$
$\newcommand{\apart}{Astropart. Phys.}$
$\newcommand{\apj}{Astrophys. J.}$
$\newcommand{\apjl}{Astrophys. J. Lett.}$
$\newcommand{\apjs}{Astrophys. J. Suppl. Ser.}$
$\newcommand{\ao}{Appl. Opt.}$
$\newcommand{\apss}{Astrophys. Space Sci.}$
$\newcommand{\aap}{Astron. Astrophys.}$
$\newcommand{\aapr}{Astron. Astrophys. Rev.}$
$\newcommand{\aaps}{Astron. Astrophys. Suppl.}$
$\newcommand{\baas}{Bull. Am. Astron. Soc.}$
$\newcommand{\caa}{Chin. Astron. Astrophys.}$
$\newcommand{\cjaa}{Chin. J. Astron. Astrophys.}$
$\newcommand{\cqg}{Class. Quantum Gravity}$
$\newcommand{\epsl}{Earth Planet. Sci. Lett.}$
$\newcommand{\expa}{Exp. Astron.}$
$\newcommand{\frass}{Front. Astron. Space Sci.}$
$\newcommand{\gal}{Galaxies}$
$\newcommand{\gca}{Geochim. Cosmochim. Acta}$
$\newcommand{\grl}{Geophys. Res. Lett.}$
$\newcommand{\icarus}{Icarus}$
$\newcommand{\ija}{Int. J. Astrobiol.}$
$\newcommand{\jatis}{J. Astron. Telesc. Instrum. Syst.}$
$\newcommand{\jcap}{J. Cosmol. Astropart. Phys.}$
$\newcommand{\jgr}{J. Geophys. Res.}$
$\newcommand{\jgrp}{J. Geophys. Res.: Planets}$
$\newcommand{\jqsrt}{J. Quant. Spectrosc. Radiat. Transf.}$
$\newcommand{\lrca}{Living Rev. Comput. Astrophys.}$
$\newcommand{\lrr}{Living Rev. Relativ.}$
$\newcommand{\lrsp}{Living Rev. Sol. Phys.}$
$\newcommand{\memsai}{Mem. Soc. Astron. Italiana}$
$\newcommand{\maps}{Meteorit. Planet. Sci.}$
$\newcommand{\mnras}{Mon. Not. R. Astron. Soc.}$
$\newcommand{\nat}{Nature}$
$\newcommand{\nastro}{Nat. Astron.}$
$\newcommand{\ncomms}{Nat. Commun.}$
$\newcommand{\ngeo}{Nat. Geosci.}$
$\newcommand{\nphys}{Nat. Phys.}$
$\newcommand{\na}{New Astron.}$
$\newcommand{\nar}{New Astron. Rev.}$
$\newcommand{\physrep}{Phys. Rep.}$
$\newcommand{\pra}{Phys. Rev. A}$
$\newcommand{\prb}{Phys. Rev. B}$
$\newcommand{\prc}{Phys. Rev. C}$
$\newcommand{\prd}{Phys. Rev. D}$
$\newcommand{\pre}{Phys. Rev. E}$
$\newcommand{\prl}{Phys. Rev. Lett.}$
$\newcommand{\psj}{Planet. Sci. J.}$
$\newcommand{\planss}{Planet. Space Sci.}$
$\newcommand{\pnas}{Proc. Natl Acad. Sci. USA}$
$\newcommand{\procspie}{Proc. SPIE}$
$\newcommand{\pasa}{Publ. Astron. Soc. Aust.}$
$\newcommand{\pasj}{Publ. Astron. Soc. Jpn}$
$\newcommand{\pasp}{Publ. Astron. Soc. Pac.}$
$\newcommand{\raa}{Res. Astron. Astrophys.}$
$\newcommand{\rmxaa}{Rev. Mexicana Astron. Astrofis.}$
$\newcommand{\sci}{Science}$
$\newcommand{\sciadv}{Sci. Adv.}$
$\newcommand{\solphys}{Sol. Phys.}$
$\newcommand{\sovast}{Soviet Astron.}$
$\newcommand{\ssr}{Space Sci. Rev.}$
$\newcommand{\uni}{Universe}$
$\newcommand{\thesection}{\Alph{section}}$</div>



<div id="title">

# Water absorption confirms cool atmospheres in two little red dots

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2602.06024-b31b1b.svg)](https://arxiv.org/abs/2602.06024)<mark>Appeared on: 2026-02-06</mark> -  _Submitted; 6 + 12 pages, 3 + 7 figures, 2 tables_

</div>
<div id="authors">

B. Wang, et al. -- incl., <mark>A. d. Graaff</mark>, <mark>R. E. Hviding</mark>, <mark>H.-W. Rix</mark>

</div>
<div id="abstract">

**Abstract:** Little red dots (LRDs) are an abundant population of compact high-redshift sources with red rest-frame optical continua, discovered by the James Webb Space Telescope (JWST) \cite{Kocevski2023,Matthee2024,Labbe2025} .Their red colors and power sources have been attributed either to dust reddening of standard hot accretion disks \cite{Furtak2024, Greene2024, Wang2024:ub} or to intrinsically cool thermal emission from dense hydrogen envelopes \cite{deGraaff2025:cliff, Naidu2025} , in both cases surrounding accreting supermassive black holes (SMBHs). These scenarios predict order-of-magnitude differences in emission temperature but have lacked decisive temperature diagnostics. Here we report a prominent absorption feature at rest-frame $\sim 1.4   \mu\mathrm{m}$ in two out of four LRDs at $z \sim 2$ with high signal-to-noise JWST spectra, among the coolest from a large LRD sample \cite{deGraaff:sample} . The feature matches the shape and wavelength of the water absorption band seen in cool stars. Atmosphere models require $T \lesssim 3000  \mathrm{K}$ to reproduce it, confirming unambiguously the presence of a cool, dense gas component contributing $20-30\%$ to the emergent continuum. A composite model reproduces both the absorption and the rest-frame optical-to-infrared continuum shape and suggests a temperature range ( $\sim2000  \mathrm{K} - 4000   \mathrm{K}$ ) rather than a single blackbody predicted by some gas envelope models \cite{Begelman2025, Kido2025, Liu2025} .Molecular absorption demonstrates that the red continua of some LRDs are intrinsic rather than dust-reddened, implying order-of-magnitude lower bolometric luminosities and black-hole masses, and providing a new diagnostic of the emitting gas.

</div>

<div id="div_fig1">

<img src="tmp_2602.06024/figs/lbol.png" alt="Fig3" width="100%"/>

**Figure 3. -** {**Bolometric luminosities and temperatures of the two cold LRDs in context of the LRD population.**}**a)** The bolometric luminosities of the sources studied here, inferred using the two-temperature stellar atmosphere model, are shown as blue and cyan stars. For context, we include the bolometric luminosities of a larger sample of LRDs derived from single temperature modified black body model fits \cite{deGraaff:sample}. For our sample blackbody fits produce practically identical bolometric luminosities (within 5-10\%).
     For comparison, the bolometric luminosities assuming a dust-reddened AGN interpretation are shown as unfilled stars of the same colors and would be at least an order magnitude higher.
     **b)** The bolometric luminosities versus wavelength at which the rest optical continuum peak for LRDs, showing only sources at $z<4.5$ where the blackbody model fits are well constrained. Longer peak wavelengths correspond to lower effective temperatures, and an approximate effective temperature conversion is shown on top. The LRDs of this paper, marked as stars, have bolometric luminosities within the range of the larger LRD population but occupy the colder side of the temperature distribution.
 (*fig:lbol*)

</div>
<div id="div_fig2">

<img src="tmp_2602.06024/figs/h2o.png" alt="Fig1" width="100%"/>

**Figure 1. -** **Detection of water absorption in two LRD spectra at $2<z<3$.**{**a)**} The two LRD JWST/$\prism$ spectra with detected water absorption are plotted in blue and cyan, respectively. Black dashed lines show power-law fits to the continuum (i.e., no water absorption) to guide the eyes. Observed M dwarf spectra with varying water absorption strength are shown in yellow and brown for comparison \cite{Filippazzo2015, Pineda2021}. We emphasize that these M dwarf spectra are not fits to the LRD data, but are included solely for illustrative purposes. The absorption features in LRDs are consistent with the characteristic water bands seen in these stellar spectra.
To quantify the absorption in a model independent way, we define a simple continuum-normalized water absorption index based on the ratio of flux densities in two narrow rest-frame windows bracketing the break at $1.310 \mu\mathrm{m}$ and $1.363 \mu\mathrm{m}$(gray vertical bands): $r_{\mathrm{H_2O}} = f\lambda,1.310/f\lambda,1.363 - 1$(i.e., more negative values indicating stronger absorption).
{**b)**} Measured water absorption strength of the LRDs (blue and cyan, shaded area indicate $1\sigma$ uncertainties) compared to $0.1 $\zsol$$\phoenix stellar models and low-density non-stellar models, across 7 orders of magnitude in densities spanning a range of temperatures. Models above $\sim 3{,}000 \mathrm{K}$ fail to produce the observed water absorption, establishing firm evidence for the existence of low temperature, $\sim 2{,}000 \mathrm{K}$, gas in LRDs. (*fig:water*)

</div>
<div id="div_fig3">

<img src="tmp_2602.06024/./figs/phoenix.png" alt="Fig2" width="100%"/>

**Figure 2. -** {**Models fits to the optical to near-infrared continua of two LRDs with detected water absorption requires cold ($T<3{,**000 \mathrm{K}$) gas.}}
    The observed JWST/NIRSpec $\prism$ spectra are shown in dark gray, with wavelength regions excluded from the fit indicated in light gray.  Single–temperature blackbody models are shown in black.
    Best-fit single-temperature blackbody models (green) broadly reproduce the continuum shape but fail to account for the depth of the water absorption feature and yield temperatures too high for the survival of water vapor.
    The best-fit two-component photospheric models are shown in orange, with the warm and cool components overplotted in blue and red, respectively. Fit residuals are shown below each panel. This minimal model reproduces both the overall continuum shape and the observed water absorption band. The inferred cool component has an effective temperature of $T\sim2{,}000 \mathrm{K}$ and contributes approximately $20\%$ and $30\%$ of the total flux at rest-frame $1.4 \mu\mathrm{m}$ in the two objects, respectively, while a very low metallicity ($Z<10^{-3} Z_\odot$) warmer component with $T\sim4{,}000$ K is required to match the optical continuum.
     (*fig:model*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2602.06024"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

121  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

21  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

5  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
